In [192]:
import pandas as pd
import os
import re
from shutil import copyfile

In [45]:
#Read files
xls = pd.ExcelFile('BIDS_info.xlsx')

#path variables
BIDS='./BIDS_data/'
NIMS='./NIMS_data/'

#Make folder if folder doesn't exist function
def makefolder(name):
    if not os.path.exists(name):
        os.makedirs(name)
        
makefolder(BIDS)




In [109]:
participants = xls.parse('participants')
protocol = xls.parse('protocol', convert_float=False).iloc[1:,:4] #columns 5 on are reference columns
protocol = protocol.dropna(axis=0, thresh=3) #get rid of items that don't have a bids equivalent
protocol.run_number = protocol.run_number.astype('str').str.strip('.0')
protocol["BIDS_scan_title_path"] = BIDS + "subprotocol.ANAT_or_FUNCT + "/sub-###_" + protocol.BIDS_scan_title + "_run-" + protocol.run_number + ".nii.gz"

NIMS_dir_ref = protocol.NIMS_scan_title.tolist()

NIMS_BIDS_conversion = protocol[[0,4]]

In [112]:
print(NIMS_BIDS_conversion)

      NIMS_scan_title                         BIDS_scan_title_path
3  BOLD_EPI_29mm_2sec   ./BIDS_data/funct/sub-###_emp_run-1.nii.gz
4  BOLD_EPI_29mm_2sec   ./BIDS_data/funct/sub-###_emp_run-2.nii.gz
5  BOLD_EPI_29mm_2sec   ./BIDS_data/funct/sub-###_emp_run-3.nii.gz
6  BOLD_EPI_29mm_2sec   ./BIDS_data/funct/sub-###_emp_run-4.nii.gz
7       T1w_9mm_BRAVO  ./BIDS_data/anat/sub-###_T1w_run-nan.nii.gz


In [90]:
#Create Participants folders in BIDS
#participants.participant_id.apply(lambda x: makefolder(BIDS + 'sub-' + str(x)))

In [39]:

def check_against_protocol(participants,protocol): 
    
    
    all_files_correct = True
    
    #Get list of files in for each participant
    for index, row in participants.iterrows():
        #Get files
        NIMS_dir_files = os.listdir(NIMS + row.nims_title)

        #Get rid of scan order headers (follow format ####_##_SCANNAME.nii.gz)
        NIMS_dir_files = [re.sub(r'[0-9]{4}_[0-9]{2}_', r'',x) for x in NIMS_dir_files]

        #Delete all non-nii.gz files
        NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]

        #Remove .nii.gz
        NIMS_dir_files = [re.sub(r'.nii.gz', r'',x) for x in NIMS_dir_files]

        #Remove files not in NIMS_dir_ref
        NIMS_dir_files = [x for x in NIMS_dir_files if x in NIMS_dir_ref]


        for item in set(NIMS_dir_ref):
            if NIMS_dir_files.count(item) == NIMS_dir_ref.count(item):
                print("sub-" + str(row.participant_id) + ": ++ " + item.rjust(20) + " match")

            else:
                print("sub-" + str(row.participant_id) + ": -- "+ item.rjust(20) + " files do not match protocol")
                all_files_correct = False
        print("------------")
        
        
    print("All your folders match your protocol") if all_files_correct else print("Some folders do not match your protocol, please resolve errors")

In [40]:
check_against_protocol(participants, protocol)

sub-100: ++   BOLD_EPI_29mm_2sec match
sub-100: ++        T1w_9mm_BRAVO match
------------
sub-101: ++   BOLD_EPI_29mm_2sec match
sub-101: ++        T1w_9mm_BRAVO match
------------
sub-102: --   BOLD_EPI_29mm_2sec files do not match protocol
sub-102: ++        T1w_9mm_BRAVO match
------------
sub-103: ++   BOLD_EPI_29mm_2sec match
sub-103: ++        T1w_9mm_BRAVO match
------------
sub-104: ++   BOLD_EPI_29mm_2sec match
sub-104: ++        T1w_9mm_BRAVO match
------------
Some folders do not match your protocol, please resolve errors


In [193]:
def rename_to_bids(participants, protocol):

    for index, row in participants.iterrows():
        #Get files
        NIMS_dir_files = os.listdir(NIMS + row.nims_title)

        #Delete all non-nii.gz files
        NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]

        for item in set(NIMS_dir_ref):
            directory_filenames = [x for x in NIMS_dir_files if item in x]
            protocol_filenames = NIMS_BIDS_conversion[NIMS_BIDS_conversion.NIMS_scan_title.str.contains(item)]
            protocol_filenames = protocol_filenames.iloc[:,1].tolist()
            
            assert len(directory_filenames) == len(protocol_filenames), "filelists aren't the same length"

            for index, item in enumerate(directory_filenames):
                oldpath = (NIMS + row.nims_title + "/" + directory_filenames[index])
                newpath = (protocol_filenames[index].replace("###", str(row.participant_id)))
                copyfile(oldpath, newpath)
        print("------------")
        


In [194]:
rename_to_bids(participants, protocol)

FileNotFoundError: [Errno 2] No such file or directory: './BIDS_data/funct/sub-100_emp_run-1.nii.gz'